<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning_LANA/blob/master/CUDA/CUDA_Jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nvidia CUDA em GPUs - Revisão

Status da GPU

In [0]:
!nvidia-smi

Mon Jan 14 15:41:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!ls

 01-check-env.py		  exemplo2.cu
 02-PyCudaWorkflow.py		  exemplo3.cu
 03-PyCudaMatrixManipulation.py   exemplo4.cu
 04-PyCudaGPUArray.py		  exemplo5.cu
 05-PyCudaElementWise.py	  exemplo6.cu
 06-PyCudaReductionKernel.py	  exemplo7.cu
 cmemory			  memory
 cmemory.cu			  memory.cu
'CUDA-Jupyter (1).ipynb'	  opatomicas
 CUDA-Jupyter.ipynb		 'opatomicas (1).cu'
 dotproduct			 'opatomicas (2).cu'
'dotproduct (1).cu'		 'opatomicas (3).cu'
 dotproduct2			  opatomicas.cu
'dotproduct (2).cu'		  pinnedmemory
'dotproduct (3).cu'		  pinnedmemory.cu
'dotproduct (4).cu'		  sample_data
 dotproduct.cu			  smemory
 Duvida-Pycuda-01.txt		  smemory.cu
 eventos			  t2est-02-PyCudaWorkflow-test.py
 eventos.cu			  test-02-PyCudaWorkflow-test.py
 exemplo1.cu


## Para importar arquivos

In [0]:
from google.colab import files
uploaded = files.upload()

Saving exemplo7.cu to exemplo7 (1).cu


## Paralelismo

Para a primeira tarefa, vamos usar os seguintes conceitos:

* <code style="color:green">&#95;&#95;global&#95;&#95;</code> - Esta palavra-chave é um qualificador usado para dizer ao compilador CUDA que a função deve ser compilada para a GPU. Para o CUDA C/C ++, o compilador nvcc irá lidar com a compilação deste código.
* <code style="color:green">blockIdx.x</code> - Esta é uma variável usada dentro de um kernel de GPU para determinar a ID do bloco que está atualmente executando o código. Uma vez que haverá muitos blocos em paralelo, precisamos desta ID para ajudar a determinar qual parte dos dados um bloco particular funcionará.
* <code style="color:green">threadIdx.x</code> - Esta é uma variável usada dentro de um kernel de GPU para determinar o ID da thread que está atualmente executando o código no bloco ativo.
* <code style="color:green">blockDim.x</code> - Esta é uma variável que retorna um valor que indica o número de threads que há por bloco. Lembre-se de que todos os blocos agendados para executar na GPU são idênticos, exceto para o valor de <code style="color:green">blockIdx.x</code>.
* <code style="color:green">myKernel <<< numero_de_blocos, threads_por_bloco>>> (...)</code> -  Esta é a sintaxe usada para iniciar um kernel na GPU. Dentro de "<<< >>>", estabelecemos dois valores. O primeiro é o número total de blocos que queremos executar na GPU, e o segundo é o número de threads que há por bloco. 

Vamos explorar os conceitos acima, fazendo um simples exemplo de "Hello Paralelismo". Ao executar a célula abaixo, teremos:

1. A partir do arquivo de origem .cu, código separado que deve ser compilado para a GPU e o código que deve ser compilado para a CPU
2. O nvcc compilará o próprio código GPU
3. nvcc dará ao compilador do host, no nosso caso gcc, o código da CPU para compilar
4. Vincula o código compilado de # 2 e # 3 e crie o executável

In [0]:
!cat exemplo1.cu

#include <stdio.h>

#define NUM_BLOCKS 16
#define BLOCK_WIDTH 1

__global__ void hello()
{
    printf("Olá! Eu sou uma thread no bloco %d\n", blockIdx.x);
}


int main(int argc,char **argv)
{
    // Inicializa o kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // Sincroniza todas as threads antes de passar o controle de volta para a CPU
    cudaDeviceSynchronize();

    printf("Processamento Concluído!\n");

    return 0;
}

In [0]:
# Compila o exemplo1 e executa o programa gerado
!nvcc -o exemplo1_out exemplo1.cu -run

Olá! Eu sou uma thread no bloco 5
Olá! Eu sou uma thread no bloco 3
Olá! Eu sou uma thread no bloco 12
Olá! Eu sou uma thread no bloco 10
Olá! Eu sou uma thread no bloco 13
Olá! Eu sou uma thread no bloco 7
Olá! Eu sou uma thread no bloco 6
Olá! Eu sou uma thread no bloco 14
Olá! Eu sou uma thread no bloco 8
Olá! Eu sou uma thread no bloco 15
Olá! Eu sou uma thread no bloco 1
Olá! Eu sou uma thread no bloco 0
Olá! Eu sou uma thread no bloco 11
Olá! Eu sou uma thread no bloco 2
Olá! Eu sou uma thread no bloco 9
Olá! Eu sou uma thread no bloco 4
Processamento Concluído!


## Inicializando um kernel na GPU - Unified Memory

In [0]:
!cat exemplo2.cu

#include <stdio.h>
#include <iostream>

// Número de elementos em cada vetor
#define N 2048 * 2048 

__global__ void my_kernel(int * a, int * b, int * c)
{
    // Determina a identificação de thread global exclusiva, por isso sabemos qual elemento processar
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if ( tid < N ) // Certifique-se de que não inicializamos mais threads do que o necessário
        c[tid] = a[tid] + b[tid];
}

void report_gpu_mem()
{
    size_t free, total;
    cudaMemGetInfo(&free, &total);
    std::cout << "Free = " << free << " Total = " << total <<std::endl;
}

int main()
{
    int *a, *b, *c;

    // Número total de bytes por vetor
    int size = N * sizeof (int); 

    // Aloca memória sem a necessidade de usar cudaMemcpy
    cudaMallocManaged(&a, size);
    cudaMallocManaged(&b, size);
    cudaMallocManaged(&c, size);

    // Inicializa memória
    for( int i = 0; i < N; ++i )
    {
        a[i] = i;
        b[i] = i;
        c[i] = 0;
    }

   

In [0]:
# Compila o exemplo2 e executa o programa gerado
!nvcc  -o exemplo2_out exemplo2.cu -run

c[4194299] = 8388598, c[4194300] = 8388600, c[4194301] = 8388602, c[4194302] = 8388604, c[4194303] = 8388606, 
Free = 11872174080 Total = 11996954624
Free = 11888951296 Total = 11996954624
Free = 11905728512 Total = 11996954624
Free = 11922505728 Total = 11996954624


### Acelerando Operações com Matrizes

In [0]:
!cat exemplo3.cu

#include <stdio.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <iostream>
#include <time.h>
using namespace std;

#define N 756

// kernel
__global__ void matrixMulGPU( int * a, int * b, int * c )
{
    int val = 0;

    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;

    if (row < N && col < N)
    {
        for ( int k = 0; k < N; ++k )
            val += a[row * N + k] * b[k * N + col];
        c[row * N + col] = val;
    }
}

void matrixMulCPU( int * a, int * b, int * c )
{
    int val = 0;

    for( int row = 0; row < N; ++row )
        for( int col = 0; col < N; ++col )
        {
            val = 0;
            for ( int k = 0; k < N; ++k )
                val += a[row * N + k] * b[k * N + col];
            c[row * N + col] = val;
        }
}

int main()
{
    int *a, *b, *c_cpu, *c_gpu;

    // Número de bytes de uma matriz N x N 
    int size = N * N * sizeof (int); 

    // Aloca memória
    c

In [0]:
# Compila o exemplo3 e executa o programa gerado
!nvcc  -o exemplo3_out exemplo3.cu -run

Tempo de processamento na GPU igual a 68.1403 msec (aproximadamente 0.01108 segundos)
Tempo de processamento na CPU igual a 1.89718 sec
Successo! As duas matrizes são iguais, sendo executadas na CPU e na GPU!


In [0]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


## Tratamento de Erro

Se você alterar consideravelmente o número de blocos e threads por bloco nos exemplos acima, você pode notar alguns casos em que você não receba a resposta esperada. Até este ponto, não adicionamos nenhum tipo de verificação de erros, o que torna muito difícil dizer por que um problema está ocorrendo. A verificação de erros é tão importante quando a programação para um GPU quanto para uma CPU. Então vamos adicionar uma verificação de erro e ver se podemos introduzir alguns erros para capturar.

**Nota**: É altamente encorajado que você inclua verificação de erros em seu código sempre que possível!

In [0]:
!cat exemplo4.cu

#include <stdio.h>

// Número de elementos em cada vetor
#define N 2048 * 2048

__global__ void my_kernel(float scalar, float * x, float * y)
{
    // Determina a identificação de thread global exclusiva, por isso sabemos qual elemento processar
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Certifique-se de que ainda temos threads disponíveis!
    if ( tid < N ) 
        y[tid] = scalar * x[tid] + y[tid];
}

int main()
{
    float *x, *y;

    // O número total de bytes por vetor
    int size = N * sizeof (float); 

    cudaError_t ierrAsync;
    cudaError_t ierrSync;

    // Aloca memória
    cudaMallocManaged(&x, size);
    cudaMallocManaged(&y, size);

    // Inicializa a memória
    for( int i = 0; i < N; ++i )
    {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    int threads_per_block = 256;
    int number_of_blocks = (N / threads_per_block) + 1;

    my_kernel <<< number_of_blocks, threads_per_block >>> ( 2.0f, x, y );

    ierrSync = cudaGetLastError();



In [0]:
# Compila o exemplo4 e executa o programa gerado
!nvcc  -o exemplo4_out exemplo4.cu -run

Max Error: 0.00000

## Consultado os Parâmetros da GPU

A API de gerenciamento de dispositivos CUDA C / C ++ permite que um programador consulte o número de dispositivos disponíveis em um sistema e os recursos de cada dispositivo. O código simples abaixo ilustra o uso da API de gerenciamento de dispositivos. Depois que o número de dispositivos habilitados para CUDA conectados ao sistema é determinado via `cudaGetDeviceCount()`, um loop sobre esses dispositivos é realizado (observe que os dispositivos são enumerados a partir de 0) e a função `cudaGetDeviceProperties()` é usada para retornar informações sobre um dispositivo em uma variável de tipo `cudaDeviceProp`. 

In [0]:
!cat exemplo5.cu

#include <stdio.h>

#define NX 200
#define NY 100

__global__ void my_kernel2D(float scalar, float * x, float * y)
{
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int col = blockIdx.y * blockDim.y + threadIdx.y;
    
    // Verifica se ainda temos threads antes de executar a operação
    if ( row < NX && col < NY ) 
        y[row * NY + col] = scalar * x[row * NY + col] + y[row * NY + col];
}

int main()
{
    float *x, *y;
    float maxError = 0;

    // Total de bytes por vetor
    int size = NX * NY * sizeof (float); 

    cudaError_t ierrAsync;
    cudaError_t ierrSync;

    cudaDeviceProp prop;

    // Aloca memória
    cudaMallocManaged(&x, size);
    cudaMallocManaged(&y, size);

    // Inicializa memória
    for( int i = 0; i < NX*NY; ++i )
    {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    dim3 threads_per_block (32,16,1);
    dim3 number_of_blocks ((NX/threads_per_block.x)+1, (NY/threads_per_block.y)+1, 1);

    cudaGetDeviceProperties(&prop, 0);
    if (th

Tente digitar tamanhos diferentes na linha de dimensão do bloco, `dim3 threads_per_block (32,16,1);` e verifique se o seu novo controle de propriedade do dispositivo GPU funciona corretamente!

À medida que você começa a escrever código de GPU que possivelmente poderia executar em múltiplos ou diferentes tipos de GPUs, você deve usar a capacidade de consultar facilmente cada dispositivo para determinar a configuração ideal para seu código.

In [0]:
# Compila o exemplo5 e executa o programa gerado
!nvcc  -o exemplo5_out exemplo5.cu -run

Max Error: 0.00000

## Gerenciamento de Memória

É importante perceber que a GPU tem sua própria memória física; Assim como a CPU usa a RAM do sistema para sua memória. Ao executar o código na GPU, temos de garantir que todos os dados necessários sejam copiados primeiro no barramento PCI-Express para a memória da GPU antes de iniciar nossos kernels.

* `cudaMalloc ( void** devPtr, size_t size )` - Esta chamada de API é usada para alocar memória na GPU, e é muito semelhante ao uso de `malloc` na CPU. Você fornece o endereço de um ponteiro que apontará para a memória após a conclusão da chamada, assim como o número de bytes a serem alocados.

* `cudaMemcpy ( void* dst, const void* src, size_t count, cudaMemcpyKind kind )` - Também é muito semelhante ao padrão `memcpy`, esta chamada API é usada para copiar dados entre a CPU e o GPU. É preciso um ponteiro de destino, um ponteiro de origem, o número de bytes a copiar e o quarto parâmetro indica qual direção os dados estão viajando: GPU-> CPU, CPU-> GPU ou GPU-> GPU.

* `cudaFree ( void* devPtr )` - Usamos essa chamada de API para liberar qualquer memória que alocamos no GPU.

* `cudaMallocManaged ( T** devPtr, size_t size );` - aloca `size` bytes na memória gerenciada e armazena em devPtr.

* `cudaFree ( void* devPtr )` - Usamos essa chamada de API para liberar qualquer memória que alocamos na memória gerenciada.

Depois de ter usado `cudaMallocManaged` para alocar alguns dados, você apenas usa o ponteiro em seu código, independentemente de ser a CPU ou a GPU acessando os dados. Antes da Memória Unificada, normalmente você tinha dois indicadores associados aos dados; Um para a memória da CPU e um para a memória do GPU (geralmente usando o nome da GPU precedido com um `d_` para indicar a memória do dispositivo).

A memória gerenciada é sincronizada entre os espaços de memória no lançamento do kernel e quaisquer pontos de sincronização do dispositivo. Isso significa que, nas arquiteturas Kepler e Maxwell, um ponto de sincronização explícito (normalmente `cudaDeviceSynchronize ()`) precisa ser inserido após um lançamento do kernel, mas antes que o host use dados gerados por esse kernel. Visite a documentação CUDA [page on Unified Memory](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#um-unified-memory-programming-hd) para mais detalhes sobre memória unificada.

In [0]:
!cat exemplo6.cu

#include <string.h>
#include <stdio.h>

struct DataElement
{
  char *name;
  int value;
};

__global__ void Kernel(DataElement *elem) {
  printf("On device: name=%s, value=%d\n", elem->name, elem->value);

  elem -> name[0] = 'd';
  elem -> value++;
}

void launch(DataElement *elem) {
  Kernel <<< 1, 1 >>> (elem);
  cudaDeviceSynchronize();
}

int main(void)
{
  DataElement *e;
  cudaMallocManaged((void**)&e, sizeof(DataElement));

  e->value = 10;
  cudaMallocManaged((void**)&(e->name), sizeof(char) * (strlen("hello") + 1) );
  strcpy(e->name, "hello");

  launch(e);

  printf("On host: name=%s, value=%d\n", e->name, e->value);

  cudaFree(e->name);
  cudaFree(e);

  cudaDeviceReset();
}

Você pode ver por que a memória unificada é atraente - ela remove o requisito de código de gerenciamento de dados complexo. Permitindo que você obtenha suas funções executando na GPU com menos esforço de desenvolvimento.

In [0]:
# Compila o exemplo6 e executa o programa gerado
!nvcc -o exemplo6_out exemplo6.cu -run

On device: name=hello, value=10
On host: name=dello, value=11


## Transposta da Matriz

Neste exemplo vamos programar um algoritmo para [Transposta da Matriz](http://en.wikipedia.org/wiki/Transpose).  Por motivos de simplicidade, usaremos matrizes quadradas. Isso nos permitirá focar as importantes técnicas de otimização de memória sem se preocupar com matrizes de forma desigual. 

O algoritmo de transposição da matriz é definido como $A_{i,j} = B_{j,i}$ onde $A$ e $B$ são $M \times M$ matrizes e os índices $i,j$ são os índices de linha e coluna, respectivamente.  (Nos exercícios de hoje vamos usar [column-major](http://en.wikipedia.org/wiki/Row-major_order#Column-major_order) para ordenação dos elementos.)

Por exemplo, se você tem um $3 \times 3$ matriz $A$ como a seguinte $$A = \left( \begin{array}{ccc}
a & d & g \\
b & e & h \\
c & f & i \end{array} \right),$$
então a transposta da matriz dado por $A^{T}$ é
$$A^{T} = \left( \begin{array}{ccc}
a & b & c \\
d & e & f \\
g & h & i \end{array} \right).$$

Este exemplo consiste em três tarefas. 

## Error Checking

Uma das técnicas de programação mais importantes para escrever código robusto é fazer uma verificação de erros adequada. Todas as funções de tempo de execução em CUDA retornam um código de erro do tipo ** `cudaError_t` **. É uma boa prática verificar o código de erro retornado de todas as funções CUDA. Neste exemplo 7, fornecemos duas macros para ajudá-lo a fazer isso. Primeiro, você pode usar `CUDA_CALL (F)` para envolver cada chamada que você faz na API de tempo de execução do CUDA. Por exemplo, em vez de escrever

```cpp
cudaMemcpy( h_c, c, sizeof(float), cudaMemcpyHostToDevice );
```

você poderia escrever

```cpp
CUDA_CALL( cudaMemcpy( h_c, c, sizeof(float), cudaMemcpyHostToDevice ) );
```

e isso irá verificar o código de retorno do `cudaMemcpy` e informá-lo se houver um erro.

Existe uma exceção para esse uso e é quando se chama kernels. Kernels não retornam nenhum valor. Para verificar se um kernel foi iniciado corretamente, você pode fazer o seguinte. Se você tiver um lançamento do kernel

```cpp
kernel<<< 256, 256 >>>( d_a, d_b, d_c );
```

você usaria a macro `CUDA_CHECK()` seguida por `CUDA_CALL( cudaDeviceSynchronize )` conforme abaixo

```cpp
kernel<<< 256, 256 >>>( d_a, d_b, d_c );
CUDA_CHECK()
CUDA_CALL( cudaDeviceSynchronize() );`
```

Nas macros de verificação de erros que fornecemos, se houver um erro, você receberá uma mensagem impressa na tela e o programa terminará. Se nenhum erro for detectado, o programa executará normalmente.

In [0]:
!cat exemplo7.cu

/*
 *  Copyright 2014 NVIDIA Corporation
 *
 *  Licensed under the Apache License, Version 2.0 (the "License");
 *  you may not use this file except in compliance with the License.
 *  You may obtain a copy of the License at
 *
 *      http://www.apache.org/licenses/LICENSE-2.0
 *
 *  Unless required by applicable law or agreed to in writing, software
 *  distributed under the License is distributed on an "AS IS" BASIS,
 *  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 *  See the License for the specific language governing permissions and
 *  limitations under the License.
 */

#include <stdio.h>

#ifdef DEBUG
#define CUDA_CALL(F)  if( (F) != cudaSuccess ) \
  {printf("Error %s at %s:%d\n", cudaGetErrorString(cudaGetLastError()), \
   __FILE__,__LINE__); exit(-1);} 
#define CUDA_CHECK()  if( (cudaPeekAtLastError()) != cudaSuccess ) \
  {printf("Error %s at %s:%d\n", cudaGetErrorString(cudaGetLastError()), \
   __FILE__,__LINE__-1); exit(-1);} 
#else
#define 

In [0]:
# Compila o exemplo7
!nvcc -lineinfo -DDEBUG -arch=sm_30 -o exemplo7_out exemplo7.cu && echo Compilado com Sucesso!

Compilado com Sucesso!


In [0]:
# Executa o exemplo7
!./exemplo7_out

Matrix size is 4096
Total memory required per matrix is 134.217728 MB
Total time CPU is 0.372804 sec
Performance is 0.720043 GB/s
Total time GPU is 0.005972 sec
Performance is 44.952137 GB/s
PASS


Se quiser gerar um zip com todos os arquivos criados, execute a célula abaixo.

In [0]:
%%bash
rm -f cuda_files.zip
zip -r cuda_files.zip . -i exemplo*.*

  adding: exemplo5.cu (deflated 63%)
  adding: exemplo4.cu (deflated 53%)
  adding: exemplo1.cu (deflated 31%)
  adding: exemplo3.cu (deflated 64%)
  adding: exemplo7.cu (deflated 65%)
  adding: exemplo2.cu (deflated 52%)
  adding: exemplo6.cu (deflated 51%)
  adding: exemplo7 (1).cu (deflated 65%)


## Download a file

In [0]:
from google.colab import files
files.download('cuda_files.zip') 

In [1]:
!ls

 cuda_files.zip   exemplo3.cu	 exemplo4_out   exemplo6.cu	   exemplo7.cu
 exemplo1.cu	  exemplo3_out	 exemplo5.cu    exemplo6_out	   exemplo7_out
 exemplo2.cu	  exemplo4.cu	 exemplo5_out  'exemplo7 (1).cu'   sample_data


** Depois de ** executar a célula acima, você pode baixar o arquivo zip [here](cuda_files.zip)